In [ ]:
#front-end UI
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>LifeUp Challenges</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            margin: 0;
            padding: 0;
            background-color: #f5f5f5;
        }

        header {
.chat-box {
         background-color: #333;
        color: white;
        padding: 1em;
        text-align: center;
    }

        main {
            padding: 2em;
        }

        .challenge {
            background-color: white;
            border: 1px solid #ddd;
            padding: 1em;
            margin-bottom: 1em;
            border-radius: 5px;
        }

        button {
            background-color: #4CAF50;
            color: white;
            padding: 10px 15px;
            border: none;
            border-radius: 5px;
            cursor: pointer;
        }

        button:hover {
            background-color: #45a049;
        }
    </style>
</head>
<body>
    <header>
        <h1>LifeUp Challenges</h1>
    </header>
    <main>
        <div id="challenges"></div>
    </main>

    <script>
        // Dummy data for challenges (replace this with actual API calls)
        const challengesData = [
            { id: 1, title: "Complete Challenge 1", points: 20 },
            { id: 2, title: "Share a Story", points: 30 },
        ];

        // Function to render challenges
        function renderChallenges(challenges) {
            const challengesContainer = document.getElementById("challenges");

            challenges.forEach(challenge => {
                const challengeElement = document.createElement("div");
                challengeElement.classList.add("challenge");
                challengeElement.innerHTML = `
                    <h3>${challenge.title}</h3>
                    <p>Points: ${challenge.points}</p>
                    <button onclick="completeChallenge(${challenge.id})">Complete</button>
                `;
                challengesContainer.appendChild(challengeElement);
            });
        }

        // Function to simulate completing a challenge (replace this with actual API calls)
        function completeChallenge(challengeId) {
            // Dummy API call (replace with actual logic)
            alert(`Challenge ${challengeId} completed!`);
        }

        // Initial rendering
        renderChallenges(challengesData);
    </script>
</body>
</html>


SyntaxError: ignored

In [ ]:
#Backend Flask
# Install required packages using: pip install flask flask-session

from flask import Flask, render_template, request, redirect, url_for, session, flash

app = Flask(__name__)
app.config['SECRET_KEY'] = 'your_secret_key'  # Change this to a secure, random key

# Dummy data for user authentication (replace this with a database)
users = [
    {"id": 1, "username": "user1", "password": "password1"},
    {"id": 2, "username": "user2", "password": "password2"},
]

# Routes

# Home page (requires authentication)
@app.route('/')
def home():
    if 'user_id' in session:
        return render_template('home.html', username=session['username'])
    else:
        return redirect(url_for('login'))

# Login page
@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        username = request.form['username']
        password = request.form['password']

        user = next((user for user in users if user["username"] == username and user["password"] == password), None)

        if user:
            session['user_id'] = user['id']
            session['username'] = user['username']
            flash('Login successful!', 'success')
            return redirect(url_for('home'))
        else:
            flash('Login failed. Check your username and password.', 'error')

    return render_template('login.html')

# Logout
@app.route('/logout')
def logout():
    session.pop('user_id', None)
    session.pop('username', None)
    flash('Logout successful!', 'success')
    return redirect(url_for('login'))

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat


In [ ]:
#home page UI
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>LifeUp Home</title>
</head>
<body>
    <h1>Welcome, {{ username }}!</h1>
    <a href="{{ url_for('logout') }}">Logout</a>
</body>
</html>

#Login Screen

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>LifeUp Login</title>
</head>
<body>
    <h1>Login</h1>
    {% with messages = get_flashed_messages() %}
        {% if messages %}
            <ul class="messages">
                {% for message in messages %}
                    <li>{{ message }}</li>
                {% endfor %}
            </ul>
        {% endif %}
    {% endwith %}
    <form method="post" action="{{ url_for('login') }}">
        <label for="username">Username:</label>
        <input type="text" id="username" name="username" required><br>
        <label for="password">Password:</label>
        <input type="password" id="password" name="password" required><br>
        <button type="submit">Login</button>
    </form>
</body>
</html>


SyntaxError: ignored

In [ ]:
#Backend (Flask with SQLAlchemy):
# Install required packages using: pip install flask flask-sqlalchemy

from flask import Flask, render_template, request, redirect, url_for, session, flash
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)
app.config['SECRET_KEY'] = 'your_secret_key'  # Change this to a secure, random key
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///lifeup.db'  # Use SQLite for simplicity
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

db = SQLAlchemy(app)

# User model
class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(50), unique=True, nullable=False)
    password = db.Column(db.String(50), nullable=False)

# Create tables
with app.app_context():
    db.create_all()

# Routes

# Home page (requires authentication)
@app.route('/')
def home():
    if 'user_id' in session:
        user = User.query.get(session['user_id'])
        return render_template('home.html', username=user.username)
    else:
        return redirect(url_for('login'))

# Login page
@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        username = request.form['username']
        password = request.form['password']

        user = User.query.filter_by(username=username, password=password).first()

        if user:
            session['user_id'] = user.id
            flash('Login successful!', 'success')
            return redirect(url_for('home'))
        else:
            flash('Login failed. Check your username and password.', 'error')

    return render_template('login.html')

# Logout
@app.route('/logout')
def logout():
    session.pop('user_id', None)
    flash('Logout successful!', 'success')
    return redirect(url_for('login'))

if __name__ == '__main__':
    app.run(debug=True)


#social media integrations

# Install required packages using: pip install flask flask-login flask-oauthlib requests

from flask import Flask, render_template, redirect, url_for, session, flash
from flask_oauthlib.client import OAuth
from flask_login import LoginManager, UserMixin, login_user, login_required, logout_user, current_user
import requests

app = Flask(__name__)
app.config['SECRET_KEY'] = 'your_secret_key'  # Change this to a secure, random key
app.config['OAUTH_TWITTER_KEY'] = 'your_twitter_consumer_key'
app.config['OAUTH_TWITTER_SECRET'] = 'your_twitter_consumer_secret'

oauth = OAuth(app)
twitter = oauth.remote_app(
    'twitter',
    consumer_key=app.config['OAUTH_TWITTER_KEY'],
    consumer_secret=app.config['OAUTH_TWITTER_SECRET'],
    request_token_params={'scope': 'email'},
    base_url='https://api.twitter.com/1.1/',
    authorize_url='https://api.twitter.com/oauth/authenticate',
    request_token_url=None,
    access_token_method='POST',
    access_token_url='https://api.twitter.com/oauth/access_token',
    authorize_params=None
)

login_manager = LoginManager(app)
login_manager.login_view = 'login'


class User(UserMixin):
    def __init__(self, id, username):
        self.id = id
        self.username = username


@login_manager.user_loader
def load_user(user_id):
    return User(int(user_id), "Test User")


# Routes

# Home page (requires authentication)
@app.route('/')
@login_required
def home():
    return render_template('home.html', username=current_user.username)


# Login page
@app.route('/login')
def login():
    return twitter.authorize(callback=url_for('twitter_authorized', _external=True))


# Logout
@app.route('/logout')
@login_required
def logout():
    logout_user()
    flash('Logout successful!', 'success')
    return redirect(url_for('login'))


# Twitter OAuth callback
@app.route('/oauth-authorized')
@twitter.authorized_handler
def twitter_authorized(resp):
    if resp is None or 'oauth_token' not in resp:
        flash('Authentication failed.', 'error')
        return redirect(url_for('login'))

    user_data = twitter.get('account/verify_credentials.json')

    user = User(user_data.data['id'], user_data.data['screen_name'])
    login_user(user)

    flash('Login successful!', 'success')
    return redirect(url_for('home'))


if __name__ == '__main__':
    app.run(debug=True)

#home html login

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>LifeUp Home</title>
</head>
<body>
    <h1>Welcome, {{ username }}!</h1>
    <a href="{{ url_for('logout') }}">Logout</a>
</body>
</html>

#home html logon

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>LifeUp Login</title>
</head>
<body>
    <h1>Login</h1>
    {% with messages = get_flashed_messages() %}
        {% if messages %}
            <ul class="messages">
                {% for message in messages %}
                    <li>{{ message }}</li>
                {% endfor %}
            </ul>
        {% endif %}
    {% endwith %}
    <a href="{{ url_for('login') }}">Login with Twitter</a>
</body>
</html>


In [ ]:
#event handling and analytics

from flask import Flask, render_template, request, redirect, url_for, session, flash
from flask_oauthlib.client import OAuth
from flask_login import LoginManager, UserMixin, login_user, login_required, logout_user, current_user

app = Flask(__name__)
app.config['SECRET_KEY'] = 'your_secret_key'  # Change this to a secure, random key

# Replace the following with your actual Google Analytics tracking code
GOOGLE_ANALYTICS_TRACKING_ID = 'your_google_analytics_tracking_id'

oauth = OAuth(app)

# Mock User class (replace with your actual User model)
class User(UserMixin):
    def __init__(self, id, username):
        self.id = id
        self.username = username


# Mock login manager (replace with your actual login manager setup)
login_manager = LoginManager(app)
login_manager.login_view = 'login'


@login_manager.user_loader
def load_user(user_id):
    return User(int(user_id), "Test User")


# Google Analytics middleware
@app.before_request
def before_request():
    if not app.debug and GOOGLE_ANALYTICS_TRACKING_ID:
        session['ga_track_id'] = GOOGLE_ANALYTICS_TRACKING_ID


# Routes

# Home page (requires authentication)
@app.route('/')
@login_required
def home():
    return render_template('home.html', username=current_user.username)


# Login page
@app.route('/login')
def login():
    return render_template('login.html')


# Logout
@app.route('/logout')
@login_required
def logout():
    logout_user()
    flash('Logout successful!', 'success')
    return redirect(url_for('login'))


if __name__ == '__main__':
    app.run(debug=True)


#html home

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>LifeUp Home</title>
    {% if 'ga_track_id' in session %}
        <!-- Google Analytics Script -->
        <script async src="https://www.googletagmanager.com/gtag/js?id={{ session['ga_track_id'] }}"></script>
        <script>
            window.dataLayer = window.dataLayer || [];
            function gtag(){dataLayer.push(arguments);}
            gtag('js', new Date());
            gtag('config', '{{ session['ga_track_id'] }}');
        </script>
    {% endif %}
</head>
<body>
    <h1>Welcome, {{ username }}!</h1>
    <!-- Example: Button Click Event -->
    <button id="exampleButton">Click me</button>

    <script>
        // Example: Button Click Event Handling
        document.getElementById('exampleButton').addEventListener('click', function() {
            // Handle button click event here
            console.log('Button clicked!');
            // You can send additional data to your analytics platform here
        });
    </script>

    <a href="{{ url_for('logout') }}">Logout</a>
</body>
</html>


#html login

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>LifeUp Login</title>
</head>
<body>
    <h1>Login</h1>
    {% with messages = get_flashed_messages() %}
        {% if messages %}
            <ul class="messages">
                {% for message in messages %}
                    <li>{{ message }}</li>
                {% endfor %}
            </ul>
        {% endif %}
    {% endwith %}
    <a href="{{ url_for('login') }}">Login with Twitter</a>
</body>
</html>
